In [0]:
%run ./01-wordcount

Out[63]: True

In [0]:
class test:
    def __init__(self):
        self.base_dir="/FileStore/project1"

    def cleanTests(self):
        print("directory cleaning started.....",end='')
        spark.sql("drop table if exists word_count_table")
        dbutils.fs.rm("/user/hive/warehouse/word_count_table", True)
        dbutils.fs.rm(f"{self.base_dir}/data/text", True)
        dbutils.fs.rm(f"{self.base_dir}/checkpoint/word-count", True)

        dbutils.fs.mkdirs(f"{self.base_dir}/data/text")
        print("Done!\n")
        print("project is setup and you can run your code now!!!!")

    def ingestData(self,itr):
        print(f"\tData ingestion started for itr_{itr}........",end='')
        dbutils.fs.cp(f"FileStore/datasets/text/text_data_{itr}.txt",f"{self.base_dir}/data/text")
        print("Done!\n")

    def assertRes(self,expectedCount):
        print("\tvalidation started...",end='')
        actual_count=spark.sql("select sum(count) from word_count_table where substr(words,1,1)=='s'").collect()[0][0]
        assert actual_count==expectedCount , f"failed tests ..expected count is {expectedCount}"
        print("done\n")

    def run_tests(self):
        self.cleanTests()
        wc=batch_count()
        self.ingestData(1)
        wc.computeBatch()
        self.assertRes(25)

        self.ingestData(2)
        wc.computeBatch()
        self.assertRes(32)

        self.ingestData(3)
        wc.computeBatch()
        self.assertRes(37)

obj=test()
obj.run_tests()
        

directory cleaning started.....Done!

project is setup and you can run your code now!!!!
	Data ingestion started for itr_1........Done!

	Executing word count...Done!

	validation started...done

	Data ingestion started for itr_2........Done!

	Executing word count...Done!

	validation started...done

	Data ingestion started for itr_3........Done!

	Executing word count...Done!

	validation started...done



In [0]:
class test_stream:
    def __init__(self):
        self.base_dir="/FileStore/project1"

    def cleanTests(self):
        print("directory cleaning started.....",end='')
        spark.sql("drop table if exists word_count_table")
        dbutils.fs.rm("/user/hive/warehouse/word_count_table", True)
        dbutils.fs.rm(f"{self.base_dir}/data/text", True)
        dbutils.fs.rm(f"{self.base_dir}/checkpoint", True)

        dbutils.fs.mkdirs(f"{self.base_dir}/data/text")
        print("Done!\n")
        print("project is setup and you can run your code now!!!!")

    def ingestData(self,itr):
        print(f"\tData ingestion started for itr_{itr}........",end='')
        dbutils.fs.cp(f"FileStore/datasets/text/text_data_{itr}.txt",f"{self.base_dir}/data/text")
        print("Done!\n")

    def assertRes(self,expectedCount):
        print("\tvalidation started...",end='')
        actual_count=spark.sql("select sum(count) from word_count_table where substr(words,1,1)=='s'").collect()[0][0]
        assert actual_count==expectedCount , f"failed tests ..expected count is {expectedCount} but the actual_count is {actual_count}"
        print("done\n")

    def run_tests(self):
        import time
        
        self.cleanTests()
        wc=stream_count()
        sQuery=wc.computeStream()
        

        self.ingestData(1)
        time.sleep(60)
        self.assertRes(25)

        self.ingestData(2)
        time.sleep(60)
        self.assertRes(32)

        self.ingestData(3)
        time.sleep(60)
        self.assertRes(37)

        sQuery.stop()


obj=test_stream()
obj.run_tests()
        

directory cleaning started.....Done!

project is setup and you can run your code now!!!!
	Executing word count...Done!

	Data ingestion started for itr_1........Done!

	validation started...done

	Data ingestion started for itr_2........Done!

	validation started...done

	Data ingestion started for itr_3........Done!

	validation started...done

